# The crypto challenge
https://cryptopals.com/

這個Challenge十分有趣，有點類似crypto的capture the flag

應該很少人學樂器、打球、繪畫認為翻翻書就可以了

程式也一樣 密碼學也一樣 所以實作吧...

ToDo: SHA 部分未完成

# Set1

### Challenge1 - Convert between encoding
雖然說是密碼學，不過第一個set大多是在處理資料轉換

> 處理 ``raw bytes`` 不要 encode 之後再處理， base64和 hex是印出來給大家看用的

用``binascii`` 這個函式庫做資料轉換

```
bytes([text[i] ^ text[32 + i] for i in range(16)]) <-> "".join([text[i] ^ text[32 + i] for i in range(16)])
bytes([text[i] ^ text[32 + i] for i in range(16)]) <-> bytearray([text[i] ^ text[32 + i] for i in range(16)])
bytes([text[i]]) <-> chr(text[i])
```

In [19]:
import Crypto
import binascii
import base64

x = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d' # raw hex
expectedY = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t' # base64
decoded = binascii.unhexlify(x) # 傳回ascii string 
print decoded
print base64.b64decode(expectedY)
print decoded.encode('hex')
print base64.b64encode(decoded)
print decoded.encode('base64') # 有帶換行
y = base64.b64encode(decoded)
if y != expectedY:
    raise Exception(y + ' != ' + expectedY)

I'm killing your brain like a poisonous mushroom
I'm killing your brain like a poisonous mushroom
49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t



In [20]:
from binascii import a2b_hex, b2a_base64
## a2b_hex == binascii.unhexlify
print a2b_hex(x) 
print b2a_base64(a2b_hex(x))

I'm killing your brain like a poisonous mushroom
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t



### Challeng2 - Fixed XOR

將兩個等長度的字串做XOR

In [21]:
from Crypto.Util.strxor import strxor

encodedS = '1c0111001f010100061a024b53535009181c' # 相同長度
encodedT = '686974207468652062756c6c277320657965'
encodedExpectedU = '746865206b696420646f6e277420706c6179'

s = binascii.unhexlify(encodedS)
t = binascii.unhexlify(encodedT)
expectedU = binascii.unhexlify(encodedExpectedU)

u = strxor(s, t)
print u
print expectedU

the kid don't play
the kid don't play


也許可用syntax sugar但crypto每一function都要小心運用

In [22]:
print hex(int(encodedS, 16)^int(encodedT, 16))
"0x%0.2X" % (int(encodedS, 16)^int(encodedT, 16))

0x746865206b696420646f6e277420706c6179L


'0x746865206B696420646F6E277420706C6179'

## Frequency of Characters
### Challenge 3 - Single-byte XOR cipher

實做我們的第一個cipher，並用著名的頻率分析攻擊法來破解，也就是說程式在求解的過程中怎樣才知道他解對了?

用英文字母的頻率來猜看看最有可能的``plaintext``

In [23]:
from Crypto.Util.strxor import strxor_c
import binascii
from Crypto.Util.strxor import strxor_c

# From http://www.data-compression.com/english.html
freqs = {
    'a': 0.0651738,
    'b': 0.0124248,
    'c': 0.0217339,
    'd': 0.0349835,
    'e': 0.1041442,
    'f': 0.0197881,
    'g': 0.0158610,
    'h': 0.0492888,
    'i': 0.0558094,
    'j': 0.0009033,
    'k': 0.0050529,
    'l': 0.0331490,
    'm': 0.0202124,
    'n': 0.0564513,
    'o': 0.0596302,
    'p': 0.0137645,
    'q': 0.0008606,
    'r': 0.0497563,
    's': 0.0515760,
    't': 0.0729357,
    'u': 0.0225134,
    'v': 0.0082903,
    'w': 0.0171272,
    'x': 0.0013692,
    'y': 0.0145984,
    'z': 0.0007836,
    ' ': 0.1918182 
}

def score(s):
    score = 0
    for i in s:
        c = i.lower()
        if c in freqs:
            score += freqs[c]
    return score

encodedS = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
s = binascii.unhexlify(encodedS)
s

'\x1b77316?x\x15\x1b\x7f+x413=x9x(7-6<x7>x:9;76'

In [24]:
def breakSingleByteXOR(s):
    curr_score = 0
    best_str = ""
    number = 0
    for i in range(0, 256):
        if(score(strxor_c(s, i)) > curr_score):
            best_str = strxor_c(s, i)
            curr_score = score(strxor_c(s, i))
            number = i
    return [number, best_str]


print breakSingleByteXOR(s)

[88, "Cooking MC's like a pound of bacon"]


### Challenge4 - Detecting the single char

接下來找出檔案中用的加密字母吧!

In [25]:
import binascii

def decodeLines(filename):
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        s = binascii.unhexlify(line)
        yield s

def findSingleByteXOR(lines):
    brokenLines = [breakSingleByteXOR(l)[1] for l in lines]
    curr_score = 0
    number = 0
    best_str = ""
    for i in range(len(brokenLines)):
        if score(brokenLines[i]) > curr_score:
            curr_score = score(brokenLines[i])
            best_str =  brokenLines[i]
    return best_str

print findSingleByteXOR(decodeLines('Text/4.txt'))

Now that the party is jumping



### Challenge 5 - Repeating-key XOR

這是一首古老的英文詩:

```
Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal
```

用 "ICE" 來對他做循環XOR 加密!

在此加密法值中，我們循序用"I"加密第一個byte，用"C"加密第二個byte，"E"加密第三個byte，再用"I"加密第四個byte...


> ord('') 得到ascii值, chr(97) 得到ascii字元

In [26]:
import binascii
from Crypto.Util.strxor import strxor_c

def encodeRepeatingKeyXor(s, key):
    return "".join([strxor_c(x[i] , ord(key[i % len(key)])) for i in range(len(x))])

x = b'''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''
key = b'ICE'
encodedExpectedY = '0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'
Y = encodeRepeatingKeyXor(x, key)
encodedY = Y.encode('hex')
print encodedY
if encodedY != encodedExpectedY:
    raise Exception(encodedY + ' != ' + encodedExpectedY)

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f


###  challenge 6 - Break the repeating XOR

challenge 6 是破解 Vigenere Cipher

1. 假設 KEYSIZE 在 2 到 40之間.
2. 寫一個 function來計算兩字串間的 edit distance/Hamming distance 例如: this is a test 和 wokka wokka!!! 的距離是37(先確認我們的function是否正確)
3. 對於每一個假設長度，我們找出最可能的key長度(看slice後的hamming distance最小者)，注意要除以 KEYSIZE 來Normalized結果!
4. 可以拿最小和次小者來試驗，或算更多 KEYSIZE block之間的hamming distance!


In [27]:
import base64
import itertools
from Crypto.Util.strxor import strxor

def getHammingDistance(x, y):
    return sum([bin(ord(x[i]) ^ ord(y[i])).count('1') for i in range(len(x))])

x = b'this is a test'
y = b'wokka wokka!!!'
expectedD = 37
d = getHammingDistance(x, y)
print d
if d != expectedD:
    raise Exception(encodedD + ' != ' + encodedExpectedD)

37


In [28]:
x = base64.b64decode(open('Text/6.txt', 'r').read())
def breakRepeatingKeyXor(x, k):
    blockstring = []
    blocks = [x[i:i+k] for i in range(0, len(x), k)]
    transposedBlocks = list(itertools.izip_longest(*blocks, fillvalue='0'))
    for i in range(len(transposedBlocks)):
        tmpblock = "".join([a for a in transposedBlocks[i]])
        blockstring.append(tmpblock)
    key = "".join([chr(breakSingleByteXOR(x)[0]) for x in blockstring])
    return key

def normalizedEditDistance(x, k):
    blocks = [x[i:i+k] for i in range(0, len(x), k)][0:4]
    pairs = list(itertools.combinations(blocks, 2))
    scores = [getHammingDistance(p[0], p[1])/float(k) for p in pairs][0:6]
    return sum(scores) / len(scores)

k = min(range(2, 41), key=lambda k: normalizedEditDistance(x, k))
k

29

5. 拿此KEYSIZE來破 
6. 接下來用此 KEYSIZE將原文slice後分成k組逐一攻破
7. 對每一組使用single-character XOR攻擊法
8. 解出所有的key

In [29]:
key = breakRepeatingKeyXor(x, k)
y = encodeRepeatingKeyXor(x, key)
print key
print y

Terminator X: Bring the noise
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take 

## AES
### Challenge8

簡單的用Pycrypto AES練習

In [30]:
import base64
from Crypto.Cipher import AES

x = base64.b64decode(open('Text/7.txt', 'r').read())

key = b'YELLOW SUBMARINE'
cipher = AES.new(key, AES.MODE_ECB)
y = cipher.decrypt(x)
print(y)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


### Challenge9

In [32]:
import itertools

lines = decodeLines('Text/8.txt')

def score(x):
    k = 16
    blocks = [x[i:i+k] for i in range(0, len(x), k)]
    pairs = itertools.combinations(blocks, 2)
    same = 0
    count = 0
    for p in pairs:
        if p[0] == p[1]:
            same += 1
    return same

lineNumber = 1
for l in lines:
    if score(l) > 0:
        print(lineNumber)
    lineNumber += 1

133


# Set 2
## Padding
### Challenge 9 - Implement PKCS#7 padding

比較下列在python 2.7的行為

In [33]:
chr(97), chr(4), str(4)

('a', '\x04', '4')

pad 任何區塊到特定長度， 在block最後說明我們pad了多長，例如:

```
"YELLOW SUBMARINE"
```
padded 到 20 bytes 

```
"YELLOW SUBMARINE\x04\x04\x04\x04"
```

另外先建立一個亂數產生器(接下來會經常用到!!)

In [36]:
from Crypto.Random import random

def to_bytes(n, length, endianess='big'):
    h = '%x' % n
    s = ('0'*(len(h) % 2) + h).zfill(length*2).decode('hex')
    return s if endianess == 'big' else s[::-1]

def randbytes(k):
    return to_bytes(random.getrandbits(8*k), k)

def padPKCS7(x, k):
    ch = k - (len(x) % k)
    return x + chr(ch) * ch


x = b'YELLOW SUBMARINE'
expectedY = b'YELLOW SUBMARINE\x04\x04\x04\x04'
y = padPKCS7(x, 20)

print y
print expectedY

if y != expectedY:
    raise Exception(y + b' != ' + expectedY)

YELLOW SUBMARINE
YELLOW SUBMARINE


In [37]:
print random.getrandbits(8*8)
randbytes(8)

4924570563036422340


'\x1e\xd6\x98A\xad\xfbP\xab'

### Challenge10 - Implement CBC mode

用ECB建構出CBC一開始的IV 用 ``all ASCII 0 (\x00\x00\x00 &c)``

![](https://i.imgur.com/dF9LU1z.png)

In [39]:
import base64
from Crypto.Cipher import AES
from Crypto.Util.strxor import strxor

class CBC:
    def __init__(self, ECB, IV):
        self._ECB = ECB
        self._IV = IV
        self._blocksize = 16

    def _getBlocks(self, s):
        return [s[i:i+self._blocksize] for i in range(0, len(s), self._blocksize)]

    def encrypt(self, plaintext):
        plainblocks = self._getBlocks(plaintext)
        ciphertext = b'' #binary string
        prev = self._IV
        for i in range(len(plainblocks)):
            plainblock = plainblocks[i]
            cipherblock = self._ECB.encrypt(strxor(plainblock, prev))
            ciphertext += cipherblock
            prev = cipherblock
        return ciphertext

    def decrypt(self, ciphertext):
        cipherblocks = self._getBlocks(ciphertext)
        plaintext = b''
        prev = self._IV
        for i in range(len(cipherblocks)):
            cipherblock = cipherblocks[i]
            plainblock = strxor(self._ECB.decrypt(cipherblock), prev)
            plaintext += plainblock
            prev = cipherblock
        return plaintext

x = base64.b64decode(open('Text/10.txt', 'r').read())
key = b'YELLOW SUBMARINE'
cipher = CBC(AES.new(key, AES.MODE_ECB), chr(0) * 16) # IV 全 ascii 0!
y = cipher.decrypt(x)
print(y)
z = cipher.encrypt(y)
if x != z:
    raise Exception(x + b' != ' + z)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


### Challenge11 - ECB, CBC Oracl

寫一個產生random 16 bytes key的function，和一個自己產生random 16 bytes key然後加密文字的function

```
encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
```

然後此function 在plaintext前後各加上 5-10 bytes，接下來寫一個function隨機使用ECB或CBC，然後我們必須去偵測此function用哪種mode!

In [40]:
from Crypto.Cipher import AES
from Crypto.Random import random

def encryption_oracle(s):
    key = randbytes(16)
    cipher = AES.new(key, AES.MODE_ECB)
    if random.randint(0, 1) == 0:
        print('Encrypting with ECB')
    else:
        print('Encrypting with CBC')
        IV = randbytes(16)
        cipher = CBC(cipher, IV)
    s = randbytes(random.randint(5, 10)) + s + randbytes(random.randint(5, 10))
    s = padPKCS7(s, 16)
    return cipher.encrypt(s)

def detectMethod(encryption_oracle):
    s = chr(0) * 47 # s = bytes([0] * 47) in python 3
    t = encryption_oracle(s)
    if t[16:32] == t[32:48]:
        return 'ECB'
    return 'CBC'

print(detectMethod(encryption_oracle))

Encrypting with CBC
CBC


## Breaking ECB
### Challenge 12 - Byte-at-a-time ECB decryption (Simple)

第二個經典的例子破解ECB，在input後加上unknown-string
```
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
```
先base64 去 deocde他，現在我們有以下型式的funtion

```
AES-128-ECB(your-string || unknown-string, random-key)
```

我們來破解這串unknown-string吧:

1. 把相同的bytes一個個餵入 function  --- 一開始先 1 byte "0", 然後 "00", 然後 "000".... 藉此找出block的size(我們不知道encoded stuff多大)。

2. 確認function是使用 ECB 加密

3. 製造出一個少1 byte(相對於block size)的input (例如若 block size 是 8 bytes，input用 "0000000") 先想看看最後一個byte要放什麼->~~

4. 建出一個look up table，涵蓋所有最後一個byted可能性的input， 例如 "00000000", "00000001", "00000002"

5. 接下來比對步驟3產生的加密結果和look up table來找出unknown-string的第一個byte

6. 重複找出下一個byte~

In [42]:
from Crypto.Cipher import AES
import base64

encodedSuffix = b'''Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK'''
key = None

def encryption_oracle(s):
    global key
    if key is None:
        key = randbytes(16)
    cipher = AES.new(key, AES.MODE_ECB)
    s = padPKCS7(s + base64.b64decode(encodedSuffix), 16)
    return cipher.encrypt(s)

def findBlockSize(encryption_oracle):
    l = len(encryption_oracle(b'')) # 利用會padded的性質
    i = 1
    while True:
        s = chr(0) * i
        t = encryption_oracle(s)
        print len(t), l
        if len(t) != l:
            return len(t) - l
        i += 1

def confirmECB(encryption_oracle, blocksize):
    s = randbytes(blocksize) * 2
    t = encryption_oracle(s)
    if t[0:blocksize] != t[blocksize:2*blocksize]:
        raise Exception('Not using ECB')

def findNextByte(encryption_oracle, blocksize, knownBytes):
    s = chr(0) * (blocksize - (len(knownBytes) % blocksize) - 1)
    d = {}
    for i in range(256):
        t = encryption_oracle(s + knownBytes + chr(i))
        d[t[0:len(s) + len(knownBytes) + 1]] = i # 建構一個dictionary，利用ECB 明文對照密文會固定的特性
    t = encryption_oracle(s)
    u = t[0:len(s) + len(knownBytes) + 1]
    if u in d:
        return d[u]
    return None

blocksize = findBlockSize(encryption_oracle)
blocksize

144 144
144 144
144 144
144 144
144 144
160 144


16

In [43]:
confirmECB(encryption_oracle, blocksize) #若不是將產生exception
s = b''
while True:
    b = findNextByte(encryption_oracle, blocksize, s)
    if b is None:
        break
    s += chr(b)
print(s)

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



### Challenge 13 - ECB cut-and-paste
### Parsing Cookie

寫一個parse k=v的程式，例如
```
foo=bar&baz=qux&zap=zazzle
```

產生似JSON的格式 
```
{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}
```

接下來寫一個可以encode user資訊的function，給他email將產生cookie

```
profile_for("foo@bar.com")

{
  email: 'foo@bar.com',
  uid: 10,
  role: 'user'
}

email=foo@bar.com&uid=10&role=user
```

functiuon需處理metacharacters(& =)，否則user可以設``foo@bar.com&role=admin``。

接下來寫兩個function

1. 加密encode後的資料，並將資料給attacker
2. 解密encode後的資料，並parse他

用 profile_for() 的input 介面 (as an oracle to generate "valid" ciphertexts) 和 ciphertexts 來產生 ``role=admin``的 profile


In [44]:
from Crypto.Cipher import AES

def encode_profile(profile):
    s = b''
    def sanitize(s):
        return s.replace(b'&', b'').replace(b'=', b'')
    for kv in profile:
        sanitizedKV = [sanitize(x.encode('ascii')) for x in kv]
        if s != b'':
            s += b'&'
        s += sanitizedKV[0] + b'=' + sanitizedKV[1]
    return s

def profile_for(email):
    profile = [
        ['email', email],
        ['uid', '10'],
        ['role', 'user']
        ]
    return encode_profile(profile)

key = randbytes(16)

def encrypt_profile_for(email):
    cipher = AES.new(key, AES.MODE_ECB)
    encoded_profile = padPKCS7(profile_for(email), 16)
    return cipher.encrypt(encoded_profile)

def unpadPKCS7(s):
    i = s[-1]
    return s[0:-ord(i)]

def decrypt_profile(s):
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_profile = unpadPKCS7(cipher.decrypt(s))
    pairs = decrypted_profile.split(b'&')
    profile = []
    for p in pairs:
        profile += [[x.decode('ascii') for x in p.split(b'=')]]
    return profile

email1 = 'foo@bar.coadmin' + ('\x0b' * 11) #直接cut and paste來取得admin的密文~~~
x1 = encrypt_profile_for(email1)
email2 = 'foo@bar.commm'
x2 = encrypt_profile_for(email2)
x = x2[0:32] + x1[16:32]
y = decrypt_profile(x)
print y

[[u'email', u'foo@bar.commm'], [u'uid', u'10'], [u'role', u'admin']]


### Challenge 14 - Byte-at-a-time ECB decryption (Harder)

多加一串 random 在前面

```
AES-128-ECB(random-prefix || attacker-controlled || target-bytes, random-key)
```

這個攻擊會變得比較困難~~

In [45]:
from Crypto.Cipher import AES
from Crypto.Random import random
import base64

key = None
prefix = None

def encryption_oracle(s):
    global key
    global prefix
    if key is None:
        key = randbytes(16)
    if prefix is None:
        # TODO(phonchi): Extend to arbitrary sizes.
        randcount = random.randint(16, 32)
        prefix = randbytes(randcount)
    cipher = AES.new(key, AES.MODE_ECB)
    s = padPKCS7(prefix + s + base64.b64decode(encodedSuffix), 16)
    return cipher.encrypt(s)

def getBlocks(s, blocksize):
    return [s[i:i+blocksize] for i in range(0, len(s), blocksize)]

def findPrefixBlock(encryption_oracle, blocksize):
    x1 = encryption_oracle(b'')
    x2 = encryption_oracle(b'0')
    blocks1 = getBlocks(x1, blocksize)
    blocks2 = getBlocks(x2, blocksize)
    for i in range(len(blocks1)):
        if blocks1[i] != blocks2[i]:
            return i

def findPrefixSizeModBlockSize(encryption_oracle, blocksize):
    def has_equal_block(blocks):
        for i in range(len(blocks) - 1):
            if blocks[i] == blocks[i+1]:
                return True
        return False

    for i in range(blocksize):
        s = chr(0) * (2*blocksize + i)
        t = encryption_oracle(s)
        blocks = getBlocks(t, blocksize)
        if has_equal_block(blocks):
            return blocksize - i

    raise Exception('Not using ECB')

def findPrefixSize(encryption_oracle, blocksize):
    return blocksize*findPrefixBlock(encryption_oracle, blocksize) + findPrefixSizeModBlockSize(encryption_oracle, blocksize)

def findNextByte(encryption_oracle, blocksize, prefixsize, knownBytes):
    k1 = blocksize - (prefixsize % blocksize)
    k2 = blocksize - (len(knownBytes) % blocksize) - 1
    k3 = prefixsize - (prefixsize % blocksize)
    s = chr(0) * (k1 + k2)
    d = {}
    for i in range(256):
        t = encryption_oracle(s + knownBytes + chr(i))
        d[t[k3+k1:k3+k1+k2 + len(knownBytes) + 1]] = i
    t = encryption_oracle(s)
    u = t[k3+k1:k3+k1+k2 + len(knownBytes) + 1]
    if u in d:
        return d[u]
    return None

blocksize = findBlockSize(encryption_oracle)
prefixsize = findPrefixSize(encryption_oracle, blocksize)
s = b''
while True:
    b = findNextByte(encryption_oracle, blocksize, prefixsize, s)
    if b is None:
        break
    s +=chr(b)
print(s)

176 176
176 176
176 176
176 176
176 176
176 176
176 176
176 176
176 176
192 176
Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



### Padding Validation
### Challenge 15 - PKCS#7 padding validation

當padding不合法時請拋出異常!!

In [46]:
s = b'ICE ICE BABY\x04\x04\x04\x04'
chr(0)

'\x00'

In [47]:
def unpadPKCS7(s):
    i = s[-1]
    if (i == chr(0)) or (s[-ord(i):] != i * ord(i)):
        raise ValueError('bad padding')
    return s[0:-ord(i)]


print(unpadPKCS7(b'ICE ICE BABY\x04\x04\x04\x04'))
try:
    unpadPKCS7(b'ICE ICE BABY\x05\x05\x05\x05')
    raise Exception('passes unexpectedly')
except ValueError:
    print "value error 1"
try:
    unpadPKCS7(b'ICE ICE BABY\x01\x02\x03\x04')
    raise Exception('passes unexpectedly')
except ValueError:
    print "value error 2"

ICE ICE BABY
value error 1
value error 2


### Challenge 16 - CBC bitflipping attacks

產生一把隨機的 AES key.

用前面的 padding 和 CBC來寫兩個 function

- 第一個 function 可以輸入任意一字串然後前面附加
```
"comment1=cooking%20MCs;userdata="
```
後面附加
```
";comment2=%20like%20a%20pound%20of%20bacon"
```
function 必須處理 ";" and "=" 字元， 且要pad 到16-byte AES 位元長 然後用該 random AES key加密


- 第二個function 去 decrypt 字串然後找看看 ";admin=true;" 有沒有在裡面，並回傳 True 或 False

用 crypto性質來破解，而不要直接從 user input 下手(第一個function 要判斷合法 input)，也就是說在不知道key的情況下來改變我們想要的ciphertext。

> CBC mode的性質為在ciphertext block的 1-bit error會:
> 1. 完全打亂該error存在的block 
> 2. 產生相同的 1-bit error 在下一個 ciphertext block.

In [48]:
from Crypto.Cipher import AES

x = ""
key = randbytes(16)
iv = randbytes(16)

def encryptParams(userdata):
    userdata = userdata.replace(';', '%3B').replace('=', '%3D')
    x1 = b'comment1=cooking%20MCs;userdata='
    x2 = b';comment2=%20like%20a%20pound%20of%20bacon'
    params = x1 + userdata.encode('ascii') + x2
    cipher = CBC(AES.new(key, AES.MODE_ECB), iv)
    return cipher.encrypt(padPKCS7(params, 16))

def decryptParamsAndCheckAdmin(encryptedParams):
    cipher = CBC(AES.new(key, AES.MODE_ECB), iv)
    paddedParams = cipher.decrypt(encryptedParams)
    params = unpadPKCS7(paddedParams)
    print params
    return params.find(b';admin=true;') != -1
encryptParams('XXXXXXXXXXXXXXXX:admin<true:XXXX')[1], len(encryptParams('XXXXXXXXXXXXXXXX:admin<true:XXXX'))
x = list(encryptParams('XXXXXXXXXXXXXXXX:admin<true:XXXX'))

In [49]:
x[32] = strxor_c(x[32], 1)
x[38] = strxor_c(x[38], 1)
x[43] = strxor_c(x[43], 1)
"".join(x)

"\x97\x98\xf4+\xe8Y5\xa5 g\xff\x9b\xdd\xbb\x04e_0p\xc5\x17'\xcfmR\xc2\xd3\x86\xc7\x145\x1c\xbe\xf0\x9d\x8b\xda-\x17\x01\xf3\xab\x97dL%C\x121~U\x08\t|\x9c\x12\xf0='\x92c\x0f\xa9e\xae\x8a$CH\x88\xde\x9e\xa4j#\xc4He+Za\xf6\xa8\xcf\xf6\xd45{\xc0\x9f/,\xfa\xc1J\xbc\xfb\xe6\xbb?\x8f\xb4\xe5n\x99\xfe\xd6\x86\xcf-c\xa8"

In [50]:
print decryptParamsAndCheckAdmin("".join(x))

comment1=cooking%20MCs;userdata=�5��ZD�۫�hs;admin=true;XXXX;comment2=%20like%20a%20pound%20of%20bacon
True


# Set 3
### Challenge 17 - CBC Padding Oracle

利用padding oracl的 side channel(對或錯來破解)

第一個function由以下的字串隨機選一個
```
MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=
MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=
MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==
MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==
MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl
MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==
MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==
MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=
MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=
MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93
```

隨機產生一個AES key存下他，將選出的string pad到16 byte AES block，用CBC加密他，並回傳ciphertext和 IV。

第二個function為server的，接收ciphertext，解密他並檢察padding，來傳回padding是否valid (這是server檢查cookie常用的方式)。


注意 byte 01h 是 valid padding，且發生機率為 1/256 在解密偽造的ciphertext時( "randomized" plaintexts情況下)

02h 02h 是 valid padding，但可能出現機率很小...

所以當我們偽造出valid padding...


In [65]:
from Crypto.Util.strxor import strxor_c
from Crypto.Util.strxor import strxor
chr(5)

'\x05'

In [66]:
from Crypto.Cipher import AES
from Crypto.Random import random
import base64
from Crypto.Util.strxor import strxor_c

strings = [
    b'MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=',
    b'MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=',
    b'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==',
    b'MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==',
    b'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl',
    b'MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==',
    b'MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==',
    b'MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=',
    b'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=',
    b'MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93'
]

key = randbytes(16)

def ciphertext_oracle():
    s = base64.b64decode(random.choice(strings))
    iv = randbytes(16)
    cipher = CBC(AES.new(key, AES.MODE_ECB), iv)
    return (iv, cipher.encrypt(padPKCS7(s, 16)))

def padding_oracle(iv, s):
    cipher = CBC(AES.new(key, AES.MODE_ECB), iv)
    paddedT = cipher.decrypt(s)
    try:
        t = unpadPKCS7(paddedT)
    except ValueError:
        return False
    return True

def decipher_last_block_previous_byte(iv, s, padding_oracle, knownI, knownP):
    k = len(knownI) + 1
    prefix = randbytes(16 - k)
    for i in range(256):
        c1 = s[-32:-16] if len(s) > 16 else iv
        if k == 16:
            c1p = chr(i) + "".join([strxor_c(ch , k) for ch in knownI])
        else:
            c1p = prefix + chr(i) + "".join([strxor_c(ch , k) for ch in knownI])            
        sp = s[:-32] + c1p + s[-16:]
        if padding_oracle(iv, sp):
            iPrev = i ^ k
            pPrev = strxor_c(c1[-k], iPrev)
            return chr(iPrev) + knownI , pPrev + knownP
    raise Exception('unexpected')

def decipher_last_block(iv, s, padding_oracle):
    knownI = b''
    knownP = b''
    for i in range(16):
        (knownI, knownP) = decipher_last_block_previous_byte(iv, s, padding_oracle, knownI, knownP)
    return knownP

def decipher(iv, s, padding_oracle):
    knownP = b''
    for i in range(len(s) / 16):
        st = s if i == 0 else s[:-i * 16]
        knownP = decipher_last_block(iv, st, padding_oracle) + knownP
    return unpadPKCS7(knownP)

(iv, s) = ciphertext_oracle()
print decipher(iv, s, padding_oracle)

000008ollin' in my five point oh


### Challenge 18 - CTR Mode

```
key=YELLOW SUBMARINE
nonce=0
format=64 bit unsigned little endian nonce,64 bit little endian block count (byte count / 16)
```

![](https://i.imgur.com/W3naT4Y.png)


CTR Mode不需padding，他只是以下列方式循序進行加密
```
keystream = AES("YELLOW SUBMARINE",
                "\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00")
keystream = AES("YELLOW SUBMARINE",
                "\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00")                
```


In [67]:
import base64
import struct
from Crypto.Cipher import AES
from Crypto.Util.strxor import strxor

class CTR:
    def __init__(self, ECB, nonce):
        self._ECB = ECB
        self._nonce = nonce
        self._blocksize = 16
        self._keybytes = b''
        self._blockcount = 0

    def encrypt(self, plaintext):
        # Work around strxor() not handling zero-length strings
        # gracefully.
        if len(plaintext) == 0:
            return b''

        keystream = self._keybytes
        while len(keystream) < len(plaintext):
            keyblock = self._ECB.encrypt(struct.pack('<QQ', self._nonce, self._blockcount))
            keystream += keyblock
            self._blockcount += 1

        if len(keystream) > len(plaintext):
            self._keybytes = keystream[len(plaintext):]
            keystream = keystream[:len(plaintext)]

        return strxor(plaintext, keystream)

    def decrypt(self, ciphertext):
        return self.encrypt(ciphertext)

x = base64.b64decode('L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==')

key = b'YELLOW SUBMARINE'
cipher1 = CTR(AES.new(key, AES.MODE_ECB), 0)
y = cipher1.decrypt(x)
print(y)
cipher2 = CTR(AES.new(key, AES.MODE_ECB), 0)
z = cipher2.encrypt(y)
if x != z:
    raise Exception(x + b' != ' + z)

Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby 


### Challenge 19 - Break fixed-nonce CTR mode using substitutions

若不randomized nonce的話，利用以下算式可破CTR

```
CIPHERTEXT-BYTE XOR PLAINTEXT-BYTE = KEYSTREAM-BYTE
```

這邊用到頻率分析中的triagram


In [68]:
from Crypto.Cipher import AES
from Crypto.Util.strxor import strxor
import base64
import binascii
import itertools

strings = [
    b'SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==',
    b'Q29taW5nIHdpdGggdml2aWQgZmFjZXM=',
    b'RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==',
    b'RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=',
    b'SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk',
    b'T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==',
    b'T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=',
    b'UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==',
    b'QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=',
    b'T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl',
    b'VG8gcGxlYXNlIGEgY29tcGFuaW9u',
    b'QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==',
    b'QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=',
    b'QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==',
    b'QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=',
    b'QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=',
    b'VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==',
    b'SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==',
    b'SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==',
    b'VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==',
    b'V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==',
    b'V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==',
    b'U2hlIHJvZGUgdG8gaGFycmllcnM/',
    b'VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=',
    b'QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=',
    b'VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=',
    b'V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=',
    b'SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==',
    b'U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==',
    b'U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=',
    b'VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==',
    b'QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu',
    b'SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=',
    b'VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs',
    b'WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=',
    b'SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0',
    b'SW4gdGhlIGNhc3VhbCBjb21lZHk7',
    b'SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=',
    b'VHJhbnNmb3JtZWQgdXR0ZXJseTo=',
    b'QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=',
]

# Generated randomly.
key = b'\xa3\xc9\xe7\xedmZU\x1e\xac\x15\xe2\xaf\xb4$\xa9{'

def encryptString(s):
    cipher = CTR(AES.new(key, AES.MODE_ECB), 0)
    return cipher.encrypt(s)

encryptedStrings = [encryptString(base64.b64decode(s)) for s in strings]

def getPrintableKeyChar(encryptedStrings, i):
    for j in range(256):
        decrypted = [strxor_c(x[i], j) for x in encryptedStrings]
        if all([x in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ,' for x in decrypted]):
            yield j # 產生iterator

def extendKey(k, ciphertext, guess):
    return k + "".join([ strxor(guess[i], ciphertext[len(k) + i]) for i in range(len(guess))])

ks = [getPrintableKeyChar(encryptedStrings, i) for i in range(10)]
ks

[<generator object getPrintableKeyChar at 0x7fcaf97d5aa0>,
 <generator object getPrintableKeyChar at 0x7fcaf97d5690>,
 <generator object getPrintableKeyChar at 0x7fcaf97d5e60>,
 <generator object getPrintableKeyChar at 0x7fcaf9771050>,
 <generator object getPrintableKeyChar at 0x7fcaf97710a0>,
 <generator object getPrintableKeyChar at 0x7fcaf97710f0>,
 <generator object getPrintableKeyChar at 0x7fcaf9771140>,
 <generator object getPrintableKeyChar at 0x7fcaf9771190>,
 <generator object getPrintableKeyChar at 0x7fcaf97711e0>,
 <generator object getPrintableKeyChar at 0x7fcaf9771230>]

In [69]:
# k = "".join(list(itertools.islice(itertools.product(*ks), 1))[0])
k = "".join([chr(strk) for strk in list(itertools.islice(itertools.product(*ks), 1))[0]])
k

'S\xe9\x95e\xc8yj\x91\xb2\x82'

In [70]:
k = extendKey(k, encryptedStrings[1], b'h ')
k = extendKey(k, encryptedStrings[3], b'entury ')
k = extendKey(k, encryptedStrings[5], b'ss ')
k = extendKey(k, encryptedStrings[3], b'se')
k = extendKey(k, encryptedStrings[5], b'rds')
k = extendKey(k, encryptedStrings[0], b' ')
k = extendKey(k, encryptedStrings[29], b'ght')
k = extendKey(k, encryptedStrings[4], b' ')
k = extendKey(k, encryptedStrings[27], b'd')
k = extendKey(k, encryptedStrings[4], b'ead')
k = extendKey(k, encryptedStrings[37], b'n,')
kl = len(k)
decrypted = [strxor(x[:kl], k[:len(x)]) + x[kl:] for x in encryptedStrings]
for i in range(len(decrypted)):
    if decrypted[i] != base64.b64decode(strings[i]):
        raise Exception('Invalid decryption')
    print(decrypted[i])

I have met them at close of day
Coming with vivid faces
From counter or desk among grey
Eighteenth-century houses.
I have passed with a nod of the head
Or polite meaningless words,
Or have lingered awhile and said
Polite meaningless words,
And thought before I had done
Of a mocking tale or a gibe
To please a companion
Around the fire at the club,
Being certain that they and I
But lived where motley is worn:
All changed, changed utterly:
A terrible beauty is born.
That woman's days were spent
In ignorant good will,
Her nights in argument
Until her voice grew shrill.
What voice more sweet than hers
When young and beautiful,
She rode to harriers?
This man had kept a school
And rode our winged horse.
This other his helper and friend
Was coming into his force;
He might have won fame in the end,
So sensitive his nature seemed,
So daring and sweet his thought.
This other man I had dreamed
A drunken, vain-glorious lout.
He had done most bitter wrong
To some who are near my heart,
Yet I number 

### Challenge 20 - Break fixed-nonce CTR statistically

用跟破repeating XOR一樣的方法

In [71]:
from Crypto.Util.strxor import strxor
import base64
import itertools

strings = [base64.b64decode(x) for x in open('Text/20.txt', 'r').read().split('\n')]
strings = strings[:-1]

encryptedStrings = [encryptString(s) for s in strings]
transposedStrings = list(zip(*encryptedStrings))
key = [chr(breakSingleByteXOR("".join(x))[0]) for x in transposedStrings]
"".join(key)

'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [72]:
def breakSameKey(strings):
    transposedStrings = list(zip(*strings))
    key = [chr(breakSingleByteXOR("".join(x))[0]) for x in transposedStrings]
    return "".join(key)
key = breakSameKey(encryptedStrings)
key

'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [73]:
key = strxor_c(encryptedStrings[0][0] , ord('I')) + key[1:]
key = extendKey(key, encryptedStrings[13], b'-M ')
key = extendKey(key, encryptedStrings[16], b'ime ')
key = extendKey(key, encryptedStrings[1], b'htnin')
key = extendKey(key, encryptedStrings[2], b'y')
key = extendKey(key, encryptedStrings[6], b'ty')
key = extendKey(key, encryptedStrings[0], b'i')
key = extendKey(key, encryptedStrings[3], b'n up')
key = extendKey(key, encryptedStrings[7], b'ession')
key = extendKey(key, encryptedStrings[4], b'or ')
key = extendKey(key, encryptedStrings[1], b'ghtenin')
key = extendKey(key, encryptedStrings[17], b'able')
key = extendKey(key, encryptedStrings[11], b'st')
key = extendKey(key, encryptedStrings[2], b'k')
key = extendKey(key, encryptedStrings[12], b'nk')
key = extendKey(key, encryptedStrings[26], b've ')
key = extendKey(key, encryptedStrings[41], b'll')
key = extendKey(key, encryptedStrings[21], b'ace')
key = extendKey(key, encryptedStrings[26], b'hole scenery')
kl = len(key)
for i in range(len(encryptedStrings)):
    s = encryptedStrings[i]
    decrypted = strxor(s[:kl], key[:len(s)]) + s[kl:]
    if decrypted != strings[i]:
        raise Exception('Invalid decryption')
    print(i, decrypted)

Exception: Invalid decryption

### Challenge 21 - MT19937

Mersenne Twister RNG

R、Python、Ruby、IDL、Free Pascal、PHP、Maple、Matlab、GMP 預設的rand()

In [74]:
class MT19937:
    def __init__(self, seed):
        self._index = 0
        self._MT = [0] * 624
        self._MT[0] = seed & 0xffffffff
        for i in range(1, 624):
            self._MT[i] = ((0x6c078965 * (self._MT[i-1] ^ (self._MT[i-1] >> 30))) + i) & 0xffffffff

    def uint32(self):
        if self._index == 0:
            self.generate_numbers()

        y = self._MT[self._index]
        y ^= (y >> 11)
        y ^= ((y << 7) & 0x9d2c5680)
        y ^= ((y << 15) & 0xefc60000)
        y ^= (y >> 18)

        self._index = (self._index + 1) % 624
        return y

    def generate_numbers(self):
        for i in range(624):
            y = (self._MT[i] & 0x80000000) + (self._MT[(i+1) % 624] & 0x7fffffff)
            self._MT[i] = self._MT[(i + 397) % 624] ^ (y >> 1)
            if y % 2 != 0:
                self._MT[i] ^= 0x9908b0df


expectedNumbers = [int(x) for x in open('Text/21.txt', 'r').read().split('\n')[:-1]]
seed = 5489
x = MT19937(seed)
for i in range(1000):
    a = x.uint32()
    if a != expectedNumbers[i]:
        raise Exception(str(i) + ' ' + a + ' != ' + expectedNumbers[i])

In [75]:
import random
random.seed(5489)
random.randint(0, 2^32)

27

### challenge 22 - Crack an MT19937 seed

In [76]:
from Crypto.Random import random
import time

t = int(time.time())
t += random.randint(40, 1000)
seed = int(t)
print(seed)
rng = MT19937(seed)
x = rng.uint32()
print(x)

t += random.randint(40, 1000)
for i in range(2000):
    k = t - i
    rng2 = MT19937(k)
    y = rng2.uint32()
    if x == y:
        print(k)

1467638123
1837720241
1467638123


### Challenge 23 - Clone an MT19937 RNG from its output

In [77]:
from Crypto.Random import random

def getMSB(x, n):
    if n < 0:
        return 0
    return (x >> (31 - n)) & 1

def setMSB(x, n, b):
    return x | (b << (31 - n))

def undoRightShiftXor(y, s):
    z = 0
    for i in range(32):
        z = setMSB(z, i, getMSB(y, i) ^ getMSB(z, i - s))
    return z

def getLSB(x, n):
    if n < 0:
        return 0
    return (x >> n) & 1

def setLSB(x, n, b):
    return x | (b << n)

def undoLeftShiftXorAnd(y, s, k):
    z = 0
    for i in range(32):
       z = setLSB(z, i, getLSB(y, i) ^ (getLSB(z, i - s) & getLSB(k, i)))
    return z

def untemper(y):
    y = undoRightShiftXor(y, 18)
    y = undoLeftShiftXorAnd(y, 15, 0xefc60000)
    y = undoLeftShiftXorAnd(y, 7, 0x9d2c5680)
    y = undoRightShiftXor(y, 11)
    return y

seed = random.getrandbits(32)
rng = MT19937(seed)
MT = [0] * 624
for i in range(624):
    MT[i] = untemper(rng.uint32())
rng2 = MT19937(0)
rng2._MT = MT

for i in range(1000):
    a = rng.uint32()
    b = rng2.uint32()
    if a != b:
        raise Exception(str(i) + ' ' + str(a) + ' != ' + str(b))

### Challenge 24 - Create the MT19937 stream cipher

In [78]:
from Crypto.Util.strxor import strxor
from Crypto.Random import random
import struct
import time

class MT19937Cipher:
    def __init__(self, key):
        self._rng = MT19937(key & 0xffff)
        self._keybytes = b''

    def encrypt(self, plaintext):
        # Work around strxor() not handling zero-length strings
        # gracefully.
        if len(plaintext) == 0:
            return b''

        keystream = self._keybytes
        while len(keystream) < len(plaintext):
            keyblock = struct.pack('<L', self._rng.uint32())
            keystream += keyblock

        if len(keystream) > len(plaintext):
            self._keybytes = keystream[len(plaintext):]
            keystream = keystream[:len(plaintext)]

        return strxor(plaintext, keystream)

    def decrypt(self, ciphertext):
        return self.encrypt(ciphertext)

key = random.getrandbits(16)

def encryption_oracle(plaintext):
    prefix = randbytes(random.randint(4, 20))
    cipher = MT19937Cipher(key)
    return cipher.encrypt(prefix + plaintext)

def recover_key(encryption_oracle):
    plaintext = b'0' * 14
    ciphertext = encryption_oracle(plaintext)
    prefix_len = len(ciphertext) - len(plaintext)
    for i in range(2**16-1):
        cipher = MT19937Cipher(i)
        s = cipher.encrypt(b'0' * len(ciphertext))
        if ciphertext[prefix_len:] == s[prefix_len:]:
            return i
    raise Exception('unexpected')

print(key, recover_key(encryption_oracle))

def token_oracle():
    seed = int(time.time())
    cipher = MT19937Cipher(seed)
    plaintext = b'0' * random.randint(4, 20)
    return cipher.encrypt(plaintext)

def is_token_for_current_time(token):
    seed = int(time.time())
    cipher = MT19937Cipher(seed)
    plaintext = b'0' * len(token)
    return cipher.encrypt(plaintext) == token

x = token_oracle()
print(x, is_token_for_current_time(x))

(48554L, 48554)
('\xb5\xfb\x01\xa3`\x1e\xf1\xd7\x80%\x93\xf9\xb4\xa4>^\xca\x81\x89', True)


# Set4
## Stream Cipher
### Challenge 25 - Break "random access read/write" AES CTR

In [79]:
from Crypto.Cipher import AES
from Crypto.Random import random
import base64
import struct

key = randbytes(16)
nonce = random.getrandbits(64)

def ciphertext_oracle():
    ecb_ciphertext = base64.b64decode(open('Text/25.txt', 'r').read())
    ecb_key = b'YELLOW SUBMARINE'
    ecb_cipher = AES.new(ecb_key, AES.MODE_ECB)
    plaintext = ecb_cipher.decrypt(ecb_ciphertext)
    cipher = CTR(AES.new(key, AES.MODE_ECB), nonce)
    return cipher.encrypt(plaintext)

def edit(ciphertext, offset, newtext):
    cipher = CTR(AES.new(key, AES.MODE_ECB), nonce)
    cipher.encrypt(b'\x00' * offset)
    return ciphertext[0:offset] + cipher.encrypt(newtext)

ciphertext = ciphertext_oracle()
plaintext = edit(ciphertext, 0, ciphertext)
print(plaintext)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


### Challenge 26 - CTR bitflipping

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import random


key = randbytes(16)
nonce = random.getrandbits(64)

def encryptParams(userdata):
    userdata = userdata.replace(';', '%3B').replace('=', '%3D')
    x1 = b'comment1=cooking%20MCs;userdata='
    x2 = b';comment2=%20like%20a%20pound%20of%20bacon'
    params = x1 + userdata.encode('ascii') + x2
    cipher = CTR(AES.new(key, AES.MODE_ECB), nonce)
    return cipher.encrypt(padPKCS7(params, 16))

def decryptParamsAndCheckAdmin(encryptedParams):
    cipher = CTR(AES.new(key, AES.MODE_ECB), nonce)
    paddedParams = cipher.decrypt(encryptedParams)
    params = unpadPKCS7(paddedParams)
    return params.find(b';admin=true;') != -1

x = list(encryptParams(':admin<true:'))
x[32] = strxor_c(x[32], 1)
x[38] = strxor_c(x[38], 1)
x[43] = strxor_c(x[43], 1)

print(decryptParamsAndCheckAdmin("".join(x)))
#x[32] ^= 1
#x[38] ^= 1
#x[43] ^= 1
#print(decryptParamsAndCheckAdmin(bytes(x)))

### Challenge 27 - Recover the key from CBC with IV=Key

為了省頻寬用 key 和 IV 相同是不安全的, Attacker 可以修改 ciphertext 來得到Key~

In [80]:
from Crypto.Cipher import AES

key = randbytes(16)

def encryptParams(userdata):
    userdata = userdata.replace(';', '%3B').replace('=', '%3D')
    x1 = b'comment1=cooking%20MCs;userdata='
    x2 = b';comment2=%20like%20a%20pound%20of%20bacon'
    params = x1 + userdata.encode('ascii') + x2
    cipher = CBC(AES.new(key, AES.MODE_ECB), key)
    return cipher.encrypt(padPKCS7(params, 16))

def decryptParamsAndCheckAdmin(encryptedParams):
    cipher = CBC(AES.new(key, AES.MODE_ECB), key)
    paddedParams = cipher.decrypt(encryptedParams)
    params = unpadPKCS7(paddedParams)
    if any([x > 127 for x in params]):
        raise ValueError(params)
    return params.find(b';admin=true;') != -1

c = encryptParams('')
c = c[0:16] + (b'\x00' * 16) + c[0:16] + c[48:]
try:
    decryptParamsAndCheckAdmin(c)
    raise Exception('unexpected')
except ValueError as e:
    text = e.args[0]
    extracted_key = "".join([strxor(text[i], text[32 + i]) for i in range(16)])
    print(extracted_key)
    if extracted_key != key:
        raise Exception(extracted_key + ' != ' + key)

!P��?ma�,��H&�


In [81]:
message = b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'
a = [ord(i) for i in message[:64]]
b = 0x67452301 ^ 0xefcdab89
b ^ 0xefcdab89

1732584193

### Challenge 28 - Implement a SHA-1 keyed MAC
To do fix it...

In [74]:
from Crypto.Util.strxor import strxor
# Taken from https://github.com/sfstpala/SlowSHA
class SHA1 (object):
    _default_h0, _default_h1, _default_h2, _default_h3, _default_h4, = (
        0x67452301, 0xefcdab89, 0x98badcfe, 0x10325476, 0xc3d2e1f0)

    def __init__(self, message, h0 = _default_h0, h1 = _default_h1, h2 = _default_h2, h3 = _default_h3, h4 = _default_h4, length = None):
        self._h0 = h0
        self._h1 = h1
        self._h2 = h2
        self._h3 = h3
        self._h4 = h4
        if length is None:
            length = len(message) * 8
        length = bin(length)[2:].rjust(64, "0")
        while len(message) > 64:
            self._handle(''.join(bin(ord(i))[2:].rjust(8, "0") for i in message[:64]))
            message = message[64:]
        message = ''.join(bin(ord(i))[2:].rjust(8, "0") for i in message) + "1"
        message += "0" * ((448 - len(message) % 512) % 512) + length
        for i in range(len(message) // 512):
            self._handle(message[i * 512:i * 512 + 512])


    def _handle(self, chunk):

        lrot = lambda x, n: (x << n) | (x >> (32 - n))
        w = []

        for j in range(len(chunk) // 32):
            w.append(int(chunk[j * 32:j * 32 + 32], 2))

        for i in range(16, 80):
            w.append(lrot(w[i - 3] ^ w[i - 8] ^ w[i - 14] ^ w[i - 16], 1)
                & 0xffffffff)

        a = self._h0
        b = self._h1
        c = self._h2
        d = self._h3
        e = self._h4
        for i in range(80):

            if i <= i <= 19:
                f, k = d ^ (b & (c ^ d)), 0x5a827999
            elif 20 <= i <= 39:
                f, k = b ^ c ^ d, 0x6ed9eba1
            elif 40 <= i <= 59:
                f, k = (b & c) | (d & (b | c)), 0x8f1bbcdc
            elif 60 <= i <= 79:
                f, k = b ^ c ^ d, 0xca62c1d6
                
            temp = lrot(a, 5) + f + e + k + w[i] & 0xffffffff
            a, b, c, d, e = temp, a, lrot(b, 30), c, d

        self._h0 = (self._h0 + a) & 0xffffffff
        self._h1 = (self._h1 + b) & 0xffffffff
        self._h2 = (self._h2 + c) & 0xffffffff
        self._h3 = (self._h3 + d) & 0xffffffff
        self._h4 = (self._h4 + e) & 0xffffffff

    def _digest(self):
        return (self._h0, self._h1, self._h2, self._h3, self._h4)

    def hexdigest(self):
        print self._digest()
        return ''.join(hex(i)[2:].rjust(8, "0") for i in self._digest())

    def digest(self):
        hexdigest = self.hexdigest()
        tmpdigest = ""
        for x in hexdigest.split('L'):
            if len(x) < 8 and x !="":
                x = '0'+ x
            tmpdigest = tmpdigest + x
        hexdigest = tmpdigest
        print hexdigest
        return bytearray(int(hexdigest[i * 2:i * 2 + 2], 16) for i in range(len(hexdigest) // 2))
        
def authSHA1(key, message):
    return SHA1(key + message).digest()

In [75]:
keylen = random.randint(0, 100)
key = randbytes(keylen)
print type(key)
message = b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'
messageDigest = authSHA1(key, message)
len(messageDigest)

<type 'str'>
(1532996819L, 126463833L, 25916121L, 2689734464L, 1872867844L)
5b5facd30789af59018b72d9a05217406fa1b204


20

### Challenge 29 - Break a SHA-1 keyed MAC using length extension

Secret-prefix SHA1 的攻擊基於你可以拿 SHA-1 的output當新一輪的input。

因為key在data前當secret-prefix，所以任何額外加入的data都會跟key有關

實作上來說，你需要考慮SHA-1是經過padded成固定的bit-length。你偽造的message需要包含padding(glue padding)

```
SHA1(key || original-message || glue-padding || new-message)
(where the final padding on the whole constructed message is implied)
```

注意要產生glue padding，你需要知道原來message的bit length，這部分需要用猜的(因為沒有key)

先實作一個function，這個function計算任何message的MD padding，並確保你的實作符合規格

接下來，偽造 SHA-1 hash並將他分為32 bit(SHA-1 calls them "a", "b", "c", &c).

讓 SHA-1 implementation 能夠傳入新的 "a", "b", "c" &c (they normally start at magic numbers)，之後偽造額外的data

用此攻擊製造出secret-prefix MAC (choose a random word from /usr/share/dict/words or something)或

```
"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"
```

偽造一個類似此的訊息結尾換成 ";admin=true".

In [68]:
from Crypto.Random import random
import struct

def padSHA1(s):
    l = len(s) * 8
    s += b'\x80'
    s += b'\x00' * ((56 - (len(s) % 64)) % 64)
    s += struct.pack('>Q', l)
    return s

keylen = random.randint(0, 100)
key = randbytes(keylen)

def validate(message, digest):
    return authSHA1(key, message) == digest

message = b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'
messageDigest = authSHA1(key, message)

def forgeHash(keylen, message, digest, suffix):
    paddedForgedMessageWithKey = padSHA1(key + message) + suffix
    forgedMessage = paddedForgedMessageWithKey[keylen:]
    h = struct.unpack('>5I', digest)
    forgedDigest = SHA1(suffix, h[0], h[1], h[2], h[3], h[4], len(paddedForgedMessageWithKey) * 8).digest()
    return (forgedMessage, forgedDigest)

def forgeValidatingHash(maxkeylen, message, digest, suffix):
    for i in range(maxkeylen):
        (forgedMessage, forgedDigest) = forgeHash(i, message, digest, suffix)
        if validate(forgedMessage, forgedDigest):
            return(forgedMessage, forgedDigest)
    raise Exception('unexpected')

print(forgeValidatingHash(100, message, messageDigest, b';admin=true'))

('comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02p;admin=true', bytearray(b'\xda\xb1\xfb6b\xcb\x01\xec\xf4N\x05\xa5\xcb#\xc9\xbaS\xc0\xa4\xbe'))


# Advance
### Challenge 31 - Implement and break HMAC-SHA1 with an artificial timing leak
Set 4 剩下來的題目是去做server的timing attack 這部分較複雜 有興趣的可以研究看看xd

寫一個小型的 application 接收 URL 且有 "file" 和 "signature" 的argument
```
http://localhost:9000/test?file=foo&signature=46b4ec586117154dacd49d664e5d63fdc88efb51
```

用server generate 出一把HMAC key，然後去驗證request的signature對file是否valid(使用``==``去實作)

接下來寫一個function，"insecure_compare", 實作 == operation 一次比較 1 byte並在碰到non-match時立即離開

在 "insecure_compare"迴圈中加入  50ms sleep (sleep 50ms after each byte) 

若MAC not Valid 回傳 500， valid則回傳200.

接下來用timing attack，寫一個程式來找出 valid MAC for any file.


In [ ]:
%%writefile challenge31.py


import binascii
import sys
import time
import urllib2


def isValidSignature(file, signature):
    start =time.clock()
    try:
        response = urllib2.urlopen('http://localhost:9000/test?file=' + file + '&signature=' + signature)
        end = time.clock()
        if response.status != 200:
            raise Exception('unexpected status ' + str(response.status))
        return (True, end - start)
    except urllib2.HTTPError as e:
        end = time.clock()
        if e.code != 500:
            raise
        return (False, end - start)

def guessNextByte(file, knownBytes, delay):
    suffixlen = 20 - len(knownBytes)
    expectedDuration = delay * len(knownBytes) + 0.01
    start = time.clock()

    def print_info(i):
        end = time.clock()
        print('Made {0} requests in {1:.2f}s; rps = {2:.2f}, max rps = {3:.2f}'.format(i, end - start, i / (end - start), 1 / expectedDuration))

    for i in range(256):
        suffix = chr(i) + (b'\x00' * (suffixlen - 1))
        signature = knownBytes + suffix
        _, duration = isValidSignature(file, binascii.hexlify(signature).decode('ascii'))
        if duration > expectedDuration + 0.8 * delay:
            print_info(i)
            print duration, expectedDuration + 0.8 * delay
            return knownBytes + chr(i)

    raise Exception('unexpected')

def guessNextByte_b(file, knownBytes, delay):
    suffixlen = 20 - len(knownBytes)
    start = time.clock()

    durations = [0] * 256

    count = 10
    for k in range(count):
        for i in range(256):
            suffix = chr(i) + (b'\x00' * (suffixlen - 1))
            signature = knownBytes + suffix
            durations[i] += isValidSignature(file, binascii.hexlify(signature).decode('ascii'))[1]

    end = time.clock()
    print('Made {0} requests in {1:.2f}s; rps = {2:.2f}'.format(count * 256, end - start, count * 256 / (end - start)))

    for i in range(256):
        durations[i] /= count

    avg_duration = sum(durations) / 256
    argmax = max(range(256), key=lambda i: durations[i])

    if durations[argmax] > avg_duration + 0.80 * delay:
        return knownBytes + chr(argmax)
    else:
        return knownBytes[:-1]

if __name__ == '__main__':
    file = sys.argv[1]
    knownBytes = b''
    DELAY = 0.5
    while True:
        if len(knownBytes) == 20:
            if isValidSignature(file, binascii.hexlify(knownBytes).decode('ascii'))[0]:
                break
            else:
                knownBytes = knownBytes[:-1]
        knownBytes = guessNextByte_b(file, knownBytes, DELAY)
        print(binascii.hexlify(knownBytes))
    print(binascii.hexlify(knownBytes))

In [ ]:
%%writefile challenge31_server.py


from Crypto.Util.strxor import strxor_c
from Crypto.Util.strxor import strxor
from Crypto.Random import random
from Crypto.Util.strxor import strxor

import binascii
import BaseHTTPServer
import SocketServer
import time
import urlparse 


def to_bytes(n, length, endianess='big'):
    h = '%x' % n
    s = ('0'*(len(h) % 2) + h).zfill(length*2).decode('hex')
    return s if endianess == 'big' else s[::-1]

def randbytes(k):
    return to_bytes(random.getrandbits(8*k), k)

PORT = 9000
DELAY = 0.05

blocksize = 64
key = randbytes(100)

# Taken from https://github.com/sfstpala/SlowSHA
class SHA1 (object):
    _default_h0, _default_h1, _default_h2, _default_h3, _default_h4, = (
        0x67452301, 0xefcdab89, 0x98badcfe, 0x10325476, 0xc3d2e1f0)

    def __init__(self, message, h0 = _default_h0, h1 = _default_h1, h2 = _default_h2, h3 = _default_h3, h4 = _default_h4, length = None):
        self._h0 = h0
        self._h1 = h1
        self._h2 = h2
        self._h3 = h3
        self._h4 = h4
        if length is None:
            length = len(message) * 8
        length = bin(length)[2:].rjust(64, "0")
        while len(message) > 64:
            self._handle(''.join(bin(ord(i))[2:].rjust(8, "0") for i in message[:64]))
            message = message[64:]
        message = ''.join(bin(ord(i))[2:].rjust(8, "0") for i in message) + "1"
        message += "0" * ((448 - len(message) % 512) % 512) + length
        for i in range(len(message) // 512):
            self._handle(message[i * 512:i * 512 + 512])


    def _handle(self, chunk):

        lrot = lambda x, n: (x << n) | (x >> (32 - n))
        w = []

        for j in range(len(chunk) // 32):
            w.append(int(chunk[j * 32:j * 32 + 32], 2))

        for i in range(16, 80):
            w.append(lrot(w[i - 3] ^ w[i - 8] ^ w[i - 14] ^ w[i - 16], 1)
                & 0xffffffff)

        a = self._h0
        b = self._h1
        c = self._h2
        d = self._h3
        e = self._h4
        for i in range(80):

            if i <= i <= 19:
                f, k = d ^ (b & (c ^ d)), 0x5a827999
            elif 20 <= i <= 39:
                f, k = b ^ c ^ d, 0x6ed9eba1
            elif 40 <= i <= 59:
                f, k = (b & c) | (d & (b | c)), 0x8f1bbcdc
            elif 60 <= i <= 79:
                f, k = b ^ c ^ d, 0xca62c1d6
                
            temp = lrot(a, 5) + f + e + k + w[i] & 0xffffffff
            a, b, c, d, e = temp, a, lrot(b, 30), c, d

        self._h0 = (self._h0 + a) & 0xffffffff
        self._h1 = (self._h1 + b) & 0xffffffff
        self._h2 = (self._h2 + c) & 0xffffffff
        self._h3 = (self._h3 + d) & 0xffffffff
        self._h4 = (self._h4 + e) & 0xffffffff

    def _digest(self):
        return (self._h0, self._h1, self._h2, self._h3, self._h4)

    def hexdigest(self):
        return ''.join(hex(i)[2:].rjust(8, "0")
            for i in self._digest())

    def digest(self):
        hexdigest = self.hexdigest()
        return bytearray(int(hexdigest[i * 2:i * 2 + 2], 16) for i in range(len(hexdigest) // 2))
        
def authSHA1(key, message):
    return SHA1(key + message).digest()

def sha1(x):
    return SHA1(x).digest()

def hmacSHA1(key, message):
    if len(key) > blocksize:
        key = sha1(key)
    if len(key) < blocksize:
        key += b'\x00' * (blocksize - len(key))

    opad = strxor_c(key, 0x5c)
    ipad = strxor_c(key, 0x36)

    return sha1(opad + sha1(ipad + message))

def insecure_compare(x, y):
    if len(x) != len(y):
        return False
    for i in range(len(x)):
        if x[i] != y[i]:
            return False
        time.sleep(DELAY)
    return True

last_file = b''

class RequestHandler(BaseHTTPServer.BaseHTTPRequestHandler):
    def do_GET(self):
        global last_file
        result = urlparse.urlparse(self.path)
        if result.path == '/test':
            q = urlparse.parse_qs(result.query)
            file = q['file'][0].encode('ascii')
            digest = hmacSHA1(key, file)
            signature = binascii.unhexlify(q['signature'][0])
            if file != last_file:
                last_file = file
                print('New file:', file, binascii.hexlify(digest))
            print(binascii.hexlify(digest), binascii.hexlify(signature))
            if insecure_compare(digest, signature):
                self.send_error(200)
            else:
                self.send_error(500)
        else:
            self.send_error(500)

def serve_forever(delay):
    global DELAY
    DELAY = delay
    SocketServer.TCPServer.allow_reuse_address = True
    httpd = SocketServer.TCPServer(("", PORT), RequestHandler)
    print("serving at port {0} with delay {1}".format(PORT, DELAY))
    httpd.serve_forever()

if __name__ == '__main__':
    serve_forever(DELAY)